In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import sweetviz as sv
import plotly.graph_objs as pg
import plotly.figure_factory as ffc
from tensorflow import keras
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
def process_data():

    train_data = pd.read_csv("weather_data.csv")
    test_data = pd.read_csv("weather_data.csv")

    #-----------TRENOVACIE DATA--------------#

    # najskor pridam do irradiance vsade 0 aby sa neodstranilo prilis vela zaznamov
    train_data["Irradiance"] = train_data["Irradiance"].fillna(0.0)

    # odstranenie prazdnych zaznamov
    train_data = train_data.dropna()
    train_data = train_data.drop_duplicates(keep='first')
    train_data = train_data.fillna(0)

    train_data = train_data.drop('Irradiance', axis=1)
    

    # zistenie kolko roznych druhov Cloud Cover sa nachadza v DB a ich zakodovnie
    # Cloud Cover sme si vybrali druh kodovanie: Label Encoding 
    unique_values = train_data['Cloud Cover'].unique()
    print(f"Unique words: {unique_values}")
    print(f"Number of unique words: {len(unique_values)}")

    train_data['Cloud Cover Encoded'] = pd.factorize(train_data['Cloud Cover'])[0]
    print(train_data.head(10))

    # na stlpec season sme vybrali druh zakódovanie One-Hot Encoding

process_data()

Unique words: ['partly cloudy' 'clear' 'overcast' 'cloudy']
Number of unique words: 4
   Temperature  Humidity  Wind Speed  Precipitation (%)    Cloud Cover  \
0         14.0      73.0         9.5               82.0  partly cloudy   
1         39.0      96.0         8.5               71.0  partly cloudy   
2         30.0      64.0         7.0               16.0          clear   
3         38.0      83.0         1.5               82.0          clear   
4         27.0      74.0        17.0               66.0       overcast   
5         32.0      55.0         3.5               26.0       overcast   
6         -2.0      97.0         8.0               86.0       overcast   
7          3.0      85.0         6.0               96.0  partly cloudy   
8          3.0      83.0         6.0               66.0       overcast   
9         28.0      74.0         8.5              107.0          clear   

   Atmospheric Pressure UV Index  Season  Visibility (km)  Location  \
0               1010.82     

In [13]:

train_data = train_data.drop('Irradiance', axis=1)
print(train_data.head(10))

NameError: name 'train_data' is not defined